# Задание 1

In [58]:
import pandas as pd
import numpy as np
import math

In [59]:
emails_df = pd.read_csv("spam_total.csv")
spam_data_df = pd.read_csv("spam_data.csv")
email_text = 'Cash Million Remove Membership Offer Money Bill'

In [60]:
emails_df

,id,SPAM,HAM
0,Emails,19,23
1,Words,121,86


In [61]:
spam_data_df

,WORD,SPAM,HAM
0,Online,7,6
1,Million,6,5
2,Bonus,4,0
3,Money,4,3
4,Unlimited,12,5
5,Offer,1,9
6,Purchase,25,8
7,Gift,1,10
8,Bill,52,34
9,Cash,9,6


In [62]:
spam_emails = emails_df.SPAM[0]
ham_emails = emails_df.HAM[0]
total_emails = spam_emails + ham_emails
total_words = spam_data_df.WORD.count()

## Вероятность спама

In [63]:
spam_pos = spam_emails / total_emails
ham_pos = ham_emails / total_emails
round(spam_pos, 3)

0.452

## Обработка сообщения

In [64]:
email_text = email_text.split()
email_text

['Cash', 'Million', 'Remove', 'Membership', 'Offer', 'Money', 'Bill']

In [65]:
email_text_array = np.array(email_text)
email_text_df = pd.DataFrame({'WORD': email_text_array})
email_text_df

,WORD
0,Cash
1,Million
2,Remove
3,Membership
4,Offer
5,Money
6,Bill


In [66]:
email_processed_df = pd.merge(email_text_df, spam_data_df, on=['WORD'], how='left').fillna(0)
email_processed_df

,WORD,SPAM,HAM
0,Cash,9.0,6.0
1,Million,6.0,5.0
2,Remove,0.0,0.0
3,Membership,0.0,0.0
4,Offer,1.0,9.0
5,Money,4.0,3.0
6,Bill,52.0,34.0


In [67]:
def spam_class():
    w_array = [num + 1 for num in email_processed_df.SPAM]
    denom = total_words + sum(spam_data_df.SPAM)
    y_spam_sum = math.log(spam_pos) + sum([math.log(w/denom) for w in w_array])
    return y_spam_sum

In [68]:
def ham_class():
    w_array = [num + 1 for num in email_processed_df.HAM]
    denom = total_words + sum(spam_data_df.HAM)
    y_ham_sum = math.log(ham_pos) + sum([math.log(w/denom) for w in w_array])
    return y_ham_sum

## Логарифм апостериорной вероятности отнесения письма к классу «спам»

In [69]:
ln_spam_pos = spam_class()
round(ln_spam_pos, 3)

-24.398

## Логарифм апостериорной вероятности отнесения письма к классу «не спам»

In [70]:
ln_ham_pos = ham_class()
round(ln_ham_pos, 3)

-21.571

##  Вероятность отнесения письма к классу «спам»

In [71]:
pos = 1 / (1 + math.exp(ln_ham_pos - ln_spam_pos))
round(pos, 3)

0.056

# Задание 2

In [187]:
knn_df = pd.read_csv("knn.csv")

In [188]:
knn_df

,id,X,Y,Class
0,1,56,82,1
1,2,96,36,0
2,3,78,73,1
3,4,69,19,1
4,5,12,44,1
5,6,77,32,0
6,7,55,38,1
7,8,86,95,1
8,9,46,99,1
9,10,15,83,0


In [189]:
point = {'X':17, 'Y':94, 'Class': None}
# point = {'X':6, 'Y':9, 'Class': None}
point

{'X': 17, 'Y': 94, 'Class': None}

In [190]:
#def euclid(given_point, dataframe):
 #       []

In [191]:
[math.sqrt((id_x - point['X'])**2 + (id_y - point['Y'])**2) for id_x, id_y in zip(knn_df.X, knn_df.Y)]

[40.80441152620633,
 98.00510190801293,
 64.51356446515724,
 91.2633551870629,
 50.24937810560445,
 86.27861844049197,
 67.67569726275453,
 69.00724599634447,
 29.427877939124322,
 11.180339887498949]

In [192]:
distances = {'id':[], 'Distance':[]}
for item_id, id_x, id_y in zip(knn_df.id,knn_df.X, knn_df.Y):
    distance = math.sqrt((id_x - point['X'])**2 + (id_y - point['Y'])**2)
    distances['id'].append(item_id)
    distances['Distance'].append(distance)

In [193]:
distances_df = pd.DataFrame(distances)
distances_df

,id,Distance
0,1,40.804412
1,2,98.005102
2,3,64.513564
3,4,91.263355
4,5,50.249378
5,6,86.278618
6,7,67.675697
7,8,69.007246
8,9,29.427878
9,10,11.180340


In [194]:
distances_full_df = pd.merge(distances_df, knn_df, on=['id'], how='inner')
distances_full_df

,id,Distance,X,Y,Class
0,1,40.804412,56,82,1
1,2,98.005102,96,36,0
2,3,64.513564,78,73,1
3,4,91.263355,69,19,1
4,5,50.249378,12,44,1
5,6,86.278618,77,32,0
6,7,67.675697,55,38,1
7,8,69.007246,86,95,1
8,9,29.427878,46,99,1
9,10,11.180340,15,83,0


In [195]:
distances_full_df.sort_values(by=['Distance'])

,id,Distance,X,Y,Class
9,10,11.180340,15,83,0
8,9,29.427878,46,99,1
0,1,40.804412,56,82,1
4,5,50.249378,12,44,1
2,3,64.513564,78,73,1
6,7,67.675697,55,38,1
7,8,69.007246,86,95,1
5,6,86.278618,77,32,0
3,4,91.263355,69,19,1
1,2,98.005102,96,36,0


In [196]:
def knn_calc(knn_type):
    distances = {'id':[], 'Distance':[]}
    for item_id, id_x, id_y in zip(knn_df.id,knn_df.X, knn_df.Y):
        if knn_type is 'euclid':
            distance = math.sqrt((id_x - point['X'])**2 + (id_y - point['Y'])**2)
        elif knn_type is 'manhattan':
            distance = abs(id_x - point['X']) + abs(id_y - point['Y'])
        else:
            return 'Введите используюмую метрику'
        distances['id'].append(item_id)
        distances['Distance'].append(round(distance, 3))
    distances_df = pd.DataFrame(distances)
    distances_full_df = pd.merge(distances_df, knn_df, on=['id'], how='inner')
    return distances_full_df.sort_values(by=['Distance']).drop(columns=['X', 'Y'])

In [197]:
knn_calc('euclid')

,id,Distance,Class
9,10,11.180,0
8,9,29.428,1
0,1,40.804,1
4,5,50.249,1
2,3,64.514,1
6,7,67.676,1
7,8,69.007,1
5,6,86.279,0
3,4,91.263,1
1,2,98.005,0


In [198]:
knn_calc('manhattan')

,id,Distance,Class
9,10,13,0
8,9,34,1
0,1,51,1
4,5,55,1
7,8,70,1
2,3,82,1
6,7,94,1
5,6,122,0
3,4,127,1
1,2,137,0
